# Data Mining Project
---

Import βιβλιοθήκες

In [ ]:
from pandas.io.sas.sas_constants import column_data_offset_offset
!pip install ucimlrepo

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from ucimlrepo import fetch_ucirepo
# fetch dataset
individual_household_electric_power_consumption = fetch_ucirepo(id=235)

Converting to dataframe

In [ ]:
# data (as pandas dataframes)
df = individual_household_electric_power_consumption.data.features
df

Καθαρισμός δεδομένων (από Object σε Numeric)

In [ ]:
cols_to_fix = [
    'Global_active_power',
    'Global_reactive_power',
    'Voltage',
    'Global_intensity',
    'Sub_metering_1',
    'Sub_metering_2',
    'Sub_metering_3'
]

df[cols_to_numeric] = df[cols_to_fix].apply(pd.to_numeric,errors='coerce')
df

Πλήθος NaN ανά στήλη

In [26]:
print(df.isna().sum())

Date                         0
Time                         0
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64
